<a href="https://colab.research.google.com/github/hucarlos08/Nerf-Geo/blob/main/PreNerf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction: The 3D Revolution and Its Neural Future

In a world where 3D models are becoming as ubiquitous as 2D images, understanding how we represent and interact with three-dimensional spaces is more critical than ever. From designing complex machinery in CAD software to simulating the behavior of structures under stress, or even immersing ourselves in virtual and augmented reality experiences, 3D representation is the backbone of countless engineering applications.

But traditional methods for representing 3D objects, like meshes and point clouds, have their limitations. They can be cumbersome, computationally expensive, and struggle to capture the nuances of complex shapes and scenes. This is where Neural Radiance Fields (NeRF), a groundbreaking approach leveraging the power of neural networks, steps in.

NeRF has revolutionized the way we think about 3D representation, offering a more flexible, efficient, and photorealistic way to capture and recreate the world around us. In this notebook, we'll embark on a journey to understand the core concepts that underpin this exciting technology. We'll start with the fundamentals of 3D representation, explore how cameras capture light, delve into the magic of rendering, and get a taste of the incredible capabilities of neural networks. By the end, you'll have a solid grasp of the key ideas that make NeRF tick, setting the stage for deeper exploration into its technical intricacies.

So, buckle up and get ready to dive into the fascinating world of 3D representation and its neural future!

**In this notebook, we'll cover the following:**

1. **Representing the 3D World:** Exploring different ways to represent objects in 3D.
2. **Capturing Light: The Pinhole Camera Model:** Understanding how cameras capture 3D scenes.
3. **Generating Images: Rendering:** Learning how to create images from 3D data.
4. **Neural Networks: Our Universal Tool:** Discovering the power of neural networks for complex tasks.
5. **View Synthesis: The NeRF Challenge:**  Understanding the problem NeRF solves and how it differs from traditional methods.
6. **NeRF Sneak Peek:** A glimpse into the inner workings and incredible results of NeRF.


## But first, necessary libraries

In [ ]:
!pip install meshio open3d



# Representing the 3D World: Beyond Lines and Dots

In the digital realm, representing three-dimensional objects accurately and efficiently is a fundamental challenge. Engineering applications, from CAD modeling to virtual reality simulations, rely heavily on our ability to capture and manipulate 3D data. Let's explore some common ways to represent 3D objects and their trade-offs:



## Meshes: Connecting the Dots

- **The Concept:** Imagine wrapping a wireframe around an object, forming a network of interconnected triangles. Each triangle is defined by three points called vertices, and the entire collection of triangles forms a mesh.
- **Applications:** Meshes are widely used in computer graphics and engineering due to their flexibility and ability to represent complex shapes.
- **Limitations:**
     - **Sharpness:** Meshes struggle to represent sharp edges or fine details accurately, requiring many tiny triangles.
     - **Complexity:** Complex models can have millions of triangles, making them computationally expensive to manipulate.

In [ ]:
import numpy as np
import plotly.graph_objects as go
import meshio
from plotly.subplots import make_subplots

!wget 'https://raw.githubusercontent.com/empet/Datasets/master/hand.off';
mymesh = meshio.read('hand.off')
verts = mymesh.points
faces = mymesh.cells_dict['triangle']
x, y, z  = verts.T
i, j, k = faces.T

#plot surface triangulation
tri_vertices = verts[faces]
Xe = []
Ye = []
Ze = []
for T in tri_vertices:
    Xe += [T[k%3][0] for k in range(4)] + [ None]
    Ye += [T[k%3][1] for k in range(4)] + [ None]
    Ze += [T[k%3][2] for k in range(4)] + [ None]


fig = make_subplots(
          rows=1, cols=1,
          subplot_titles=('3d Mesh', 'Mesh3d with vertex intensities', 'Mesh3d with cell intensities'),
          horizontal_spacing=0.02,
          specs=[[{"type": "scene"}]])

fig.add_trace(go.Scatter3d(x=Xe,
                     y=Ye,
                     z=Ze,
                     mode='lines',
                     name='',
                     line=dict(color= 'rgb(40,40,40)', width=0.5)), 1, 1)


fig.update_layout(width=600, height=600, font_size=10)
fig.update_scenes(camera_eye_x=1.45, camera_eye_y=1.45, camera_eye_z=1.45)

## Point Clouds: A Scattered Representation

[Image of a 3D point cloud model]

* **The Concept:** A point cloud is simply a collection of points in 3D space, each with coordinates (x, y, z). Imagine a swarm of fireflies representing the surface of an object.
* **Applications:** Point clouds are often generated from LiDAR (Light Detection and Ranging) scans, which measure distances to objects using lasers.
* **Limitations:**
    * **No Connectivity:** Point clouds lack information about how points are connected, making it difficult to distinguish the object's surface from empty space.
    * **Rendering:** Creating realistic images from point clouds is challenging due to the lack of surface information

In [ ]:
import open3d as o3d

#draw a point cloud with default parameter
dataset = o3d.data.PLYPointCloud()
office = o3d.io.read_point_cloud(dataset.path)
o3d.visualization.draw_plotly([office])

## Volumetric Representation: Filling the Space

* **The Concept:**  Imagine dividing 3D space into tiny cubes called voxels (like 3D pixels). Each voxel stores information about the density and color of the material it represents.
* **Advantages:**
    * **Continuous:** Volumetric representations capture the continuous nature of real-world objects, unlike discrete meshes or points.
    * **Implicit Surfaces:**  We can define surfaces implicitly as regions where density crosses a certain threshold.
* **Relevance to NeRF:** Neural Radiance Fields leverage this volumetric representation, learning the density and color of each point in space. This allows for incredibly detailed and realistic 3D scenes.

In [ ]:
# Import data
import time
import numpy as np

from skimage import io

vol = io.imread("https://s3.amazonaws.com/assets.datacamp.com/blog_assets/attention-mri.tif")
volume = vol.T
r, c = volume[0].shape

# Define frames
import plotly.graph_objects as go
nb_frames = 68

fig = go.Figure(frames=[go.Frame(data=go.Surface(
    z=(6.7 - k * 0.1) * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67 - k]),
    cmin=0, cmax=200
    ),
    name=str(k) # you need to name the frame for the animation to behave properly
    )
    for k in range(nb_frames)])

# Add data to be displayed before animation starts
fig.add_trace(go.Surface(
    z=6.7 * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67]),
    colorscale='Gray',
    cmin=0, cmax=200,
    colorbar=dict(thickness=20, ticklen=4)
    ))


def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout
fig.update_layout(
         title='Slices in volumetric data',
         width=600,
         height=600,
         scene=dict(
                    zaxis=dict(range=[-0.1, 6.8], autorange=False),
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(50)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

fig.show()